In [1]:
import pandas as pd
import csv
import numpy as np
import scipy
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=40,20
import matplotlib.mlab as mlab
from IPython.display import display
pd.options.display.max_columns = None
from numpy import random
from statsmodels import robust
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import itertools
import networkx as nx
import datetime


In [2]:
ipsc_samples_df_short = pd.read_csv('./iPSC/ipsc_samples_df_short.csv')
ipsc_samples_df_short.shape

(6330, 320)

In [3]:
gtex_shortlisted = pd.read_csv("./iPSC/gtex_shortlisted_ipsc.tsv",sep="\t")
gtex_shortlisted.shape

(6380, 11690)

In [4]:
ipsc_samples_genes = ipsc_samples_df_short['Gene_name'].drop_duplicates().tolist()
print(len(ipsc_samples_genes))
gtex_shortlisted = gtex_shortlisted[gtex_shortlisted['Gene_name'].isin(ipsc_samples_genes)]
gtex_shortlisted.shape

6330


(6330, 11690)

In [6]:
sampleid_mapping = pd.read_csv('./data_generated/GTEx_Tissue_Attributes.csv',sep='\t')
sampleid_mapping = sampleid_mapping[['SAMPID','SMTS']]
print(sampleid_mapping.shape)
sampleid_mapping.head(4)

(22951, 2)


,SAMPID,SMTS
0,GTEX-1117F-0003-SM-58Q7G,Blood
1,GTEX-1117F-0003-SM-5DWSB,Blood
2,GTEX-1117F-0003-SM-6WBT7,Blood
3,GTEX-1117F-0011-R10a-SM-AHZ7F,Brain


In [7]:
ipsc_samples_df_short = ipsc_samples_df_short.drop(columns = ['Ensembl_id','transcript_length'])
ipsc_samples_df_short.columns

Index(['GSM2100046_291_1_counts', 'GSM2099989_107_5_counts',
       'GSM2100039_255_2_counts', 'GSM2100240_845_6_counts',
       'GSM2100242_864_4_counts', 'GSM2100193_821_8_counts',
       'GSM2100273_998_4_counts', 'GSM2099965_024_1_counts',
       'GSM2100069_352_1_counts', 'GSM2100008_178_2_counts',
       ...
       'GSM2100064_345_1_counts', 'GSM2100032_243_1_counts',
       'GSM2099966_024_4_counts', 'GSM2100230_839_5_counts',
       'GSM2100170_601_4_counts', 'GSM2100107_799_03_counts',
       'GSM2100182_668_1_counts', 'GSM2099968_054_1_counts',
       'GSM2100023_215_1_counts', 'Gene_name'],
      dtype='object', length=318)

In [8]:
ipsc_samples = ipsc_samples_df_short.columns
for i in ipsc_samples:
    if i not in ['Gene','transcript_length','Gene_name']:
        sampleid_mapping = sampleid_mapping.append({'SAMPID' : i , 'SMTS' : 'iPSC'} , ignore_index=True)
    
sampleid_mapping.shape

(23268, 2)

In [9]:
sampleid_mapping.tail()

,SAMPID,SMTS
23263,GSM2100170_601_4_counts,iPSC
23264,GSM2100107_799_03_counts,iPSC
23265,GSM2100182_668_1_counts,iPSC
23266,GSM2099968_054_1_counts,iPSC
23267,GSM2100023_215_1_counts,iPSC


In [10]:
sampleid_mapping_dict = {}
for index, row in sampleid_mapping.iterrows():
    key  = row["SAMPID"]
    val = row['SMTS']
    #print(row["SAMPID"])
    sampleid_mapping_dict[key] = val

In [11]:
print(ipsc_samples_df_short.shape)
print(gtex_shortlisted.shape)

(6330, 318)
(6330, 11690)


In [12]:
gtex_shortlisted = pd.merge(gtex_shortlisted,ipsc_samples_df_short,how = 'left',on='Gene_name')
gtex_shortlisted = gtex_shortlisted.drop(columns=['Ensembl_id'])
gtex_shortlisted.shape

(6330, 12006)

In [13]:
gtex_shortlisted.head(1)

,Gene_name,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2226-SM-5N9CH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,GTEX-1117F-3026-SM-5GZYU,GTEX-1117F-3226-SM-5N9CT,GTEX-111CU-0126-SM-5GZWZ,GTEX-111CU-0226-SM-5GZXC,GTEX-111CU-0326-SM-5GZXO,GTEX-111CU-0426-SM-5GZY1,GTEX-111CU-0526-SM-5EGHK,GTEX-111CU-0626-SM-5EGHL,GTEX-111CU-0726-SM-5GZYD,GTEX-111CU-0826-SM-5EGIJ,GTEX-111CU-0926-SM-5EGIK,GTEX-111CU-1026-SM-5EGIL,GTEX-111CU-1126-SM-5EGIM,GTEX-111CU-1226-SM-5EGIN,GTEX-111CU-1326-SM-5NQ8L,GTEX-111CU-1426-SM-5GZYP,GTEX-111CU-1526-SM-5N9FS,GTEX-111CU-1726-SM-5EGHM,GTEX-111CU-1826-SM-5GZYN,GTEX-111CU-2026-SM-5GZZC,GTEX-111CU-2226-SM-5N9G5,GTEX-111FC-0126-SM-5N9DL,GTEX-111FC-0226-SM-5N9B8,GTEX-111FC-0326-SM-5GZZ1,GTEX-111FC-0426-SM-5N9CV,GTEX-111FC-0526-SM-5GZZ8,GTEX-111FC-0626-SM-5N9CU,GTEX-111FC-0826-SM-5GZWO,GTEX-111FC-1026-SM-5GZX1,GTEX-111FC-1126-SM-5GZWU,GTEX-111FC-1326-SM-5N9D9,GTEX-111FC-1926-SM-5GZYC,GTEX-111FC-2026-SM-5GZYO,GTEX-111FC-2526-SM-5GZXU,GTEX-111FC-2926-SM-5GZY7,GTEX-111FC-3126-SM-5GZZ2,GTEX-111FC-3326-SM-5GZYV,GTEX-111VG-0008-SM-5Q5BG,GTEX-111VG-0326-SM-5GZX7,GTEX-111VG-0526-SM-5N9BW,GTEX-111VG-0726-SM-5GIDC,GTEX-111VG-1626-SM-5EGIO,GTEX-111VG-1926-SM-5GIDO,GTEX-111VG-2226-SM-5N9DU,GTEX-111VG-2326-SM-5N9BK,GTEX-111VG-2426-SM-5GZXD,GTEX-111VG-2526-SM-5GZXP,GTEX-111VG-2626-SM-5GZY2,GTEX-111YS-0006-SM-5NQBE,GTEX-111YS-0008-SM-5Q5BH,GTEX-111YS-0126-SM-5987T,GTEX-111YS-0326-SM-5GZZ3,GTEX-111YS-0426-SM-5987O,GTEX-111YS-0526-SM-5GZXJ,GTEX-111YS-0626-SM-5GZXV,GTEX-111YS-0726-SM-5GZY8,GTEX-111YS-0826-SM-5GZYK,GTEX-111YS-0926-SM-5EGGI,GTEX-111YS-1026-SM-5GZYE,GTEX-111YS-1126-SM-5GZYQ,GTEX-111YS-1226-SM-5EGGJ,GTEX-111YS-1326-SM-5EGGK,GTEX-111YS-1426-SM-5GID8,GTEX-111YS-1526-SM-5GZYW,GTEX-111YS-1626-SM-5GZZ9,GTEX-111YS-1726-SM-5GIED,GTEX-111YS-1826-SM-5GIEP,GTEX-111YS-1926-SM-5GICC,GTEX-111YS-2026-SM-5EGGL,GTEX-111YS-2126-SM-5EGGM,GTEX-111YS-2226-SM-5987P,GTEX-111YS-2326-SM-5987L,GTEX-111YS-2426-SM-5GZZQ,GTEX-1122O-0003-SM-5Q5DL,GTEX-1122O-0005-SM-5O99J,GTEX-1122O-0008-SM-5QGR2,GTEX-1122O-0126-SM-5GICA,GTEX-1122O-0226-SM-5N9DA,GTEX-1122O-0326-SM-5H124,GTEX-1122O-0426-SM-5H12G,GTEX-1122O-0526-SM-5N9DM,GTEX-1122O-0626-SM-5N9B9,GTEX-1122O-0726-SM-5GIEV,GTEX-1122O-0826-SM-5GICV,GTEX-1122O-0926-SM-5N9C9,GTEX-1122O-1126-SM-5NQ8X,GTEX-1122O-1226-SM-5H113,GTEX-1122O-1326-SM-5H11F,GTEX-1122O-1426-SM-5H11R,GTEX-1122O-1526-SM-5N9CL,GTEX-1122O-1626-SM-5N9CX,GTEX-1122O-1726-SM-5GICM,GTEX-1122O-1826-SM-5EGIP,GTEX-1122O-1926-SM-5EGIQ,GTEX-1122O-2026-SM-5NQ91,GTEX-1122O-2126-SM-5EGIR,GTEX-1122O-2426-SM-5GIDN,GTEX-1128S-0005-SM-5P9HI,GTEX-1128S-0008-SM-5Q5DP,GTEX-1128S-0126-SM-5H12S,GTEX-1128S-0326-SM-5GZZF,GTEX-1128S-0726-SM-5N9D6,GTEX-1128S-0826-SM-5GZZI,GTEX-1128S-0926-SM-5GZZU,GTEX-1128S-1626-SM-5H12O,GTEX-1128S-2126-SM-5H12U,GTEX-1128S-2326-SM-5GZZY,GTEX-1128S-2426-SM-5H11B,GTEX-1128S-2526-SM-5H11N,GTEX-1128S-2626-SM-5H11Z,GTEX-1128S-2726-SM-5H12C,GTEX-1128S-2826-SM-5N9DI,GTEX-113IC-0006-SM-5NQ9C,GTEX-113IC-0008-SM-5QGRF,GTEX-113IC-0126-SM-5HL6T,GTEX-113IC-0226-SM-5HL5C,GTEX-113IC-0426-SM-5HL5O,GTEX-113IC-0526-SM-5HL76,GTEX-113JC-0006-SM-5O997,GTEX-113JC-0008-SM-5QGR6,GTEX-113JC-0126-SM-5EGJW,GTEX-113JC-0726-SM-5GZZR,GTEX-113JC-0926-SM-5H114,GTEX-113JC-1026-SM-5H117,GTEX-113JC-1926-SM-5N9GE,GTEX-113JC-2226-SM-5EGJG,GTEX-113JC-2326-SM-5EQ4E,GTEX-113JC-2726-SM-5EGIS,GTEX-113JC-2826-SM-5EGIT,GTEX-117XS-0005-SM-5PNU6,GTEX-117XS-0008-SM-5Q5DQ,GTEX-117XS-0426-SM-5GZZN,GTEX-117XS-0526-SM-5987Q,GTEX-117XS-0726-SM-5H131,GTEX-117XS-1926-SM-5GICO,GTEX-117XS-2026-SM-5GID1,GTEX-117XS-2226-SM-5GIE1,GTEX-117XS-2526-SM-5H11G,GTEX-117XS-2726-SM-5N9BL,GTEX-117XS-2926-SM-5N9BX,GTEX-117XS-3026-SM-5N9CA,GTEX-117XS-3126-SM-5GIDP,GTEX-117YW-0005-SM-5NQ8Z,GTEX-117YW-0008-SM-5QGRD,GTEX-117YW-0126-SM-5EGGN,GTEX-117YW-0226-SM-5N9CM,GTEX-117YW-0326-SM-5N9CY,GTEX-117YW-0426-SM-5GZZZ,GTEX-117YW-052

In [14]:
gtex_shortlisted.isnull().values.any()


False

In [15]:
def nonzero_data(dataset):
    nz_dataset = []
    nonzero_indices = np.flatnonzero(dataset)
    for i in nonzero_indices:
        nz_dataset.append(dataset[i])
    return nz_dataset
    
#not removing top and bottom 5% of data or anything ~~~
def gene_level_attributes_function(dataset):
    gene_exp_dict = {}    
    gene_list = []
    mean_val = []
    std_dev = []
    min_val = []
    max_val = []
    range_val = []
    IQR = []
    median_val = []
    median_wo_zeros = []
    percentile_25 = []
    percentile_40 = []
    percentile_75 = []
    CV = []
    count = 0
    for row in dataset.iterrows():
        count+=1
        index,info = row
        gene = list(info)[0]
        data = list(info)[1:]
        data = np.array(data).astype(np.float)
        #print(data[1])
        gene_list.append(gene)
        gene_exp_dict[gene] = data
        mean_val.append(np.mean(data))
        std_dev.append(np.std(data))
        min_val.append(np.amin(data))
        max_val.append(np.amax(data))
        range_val.append(np.amax(data) - np.amin(data))
        median_val.append(np.median(data))
        median_wo_zeros.append(np.median(nonzero_data(data)))
        percentile_25.append(np.percentile(data, 25))
        percentile_40.append(np.percentile(data, 40))
        percentile_75.append(np.percentile(data, 75))
        IQR.append(np.percentile(data, 75) - np.percentile(data, 25))
        CV.append(np.std(data)/ np.mean(data))        
        if count%100==0:
            print(count)
            
    d = {'Gene': gene_list, 'MeanExp' : mean_val, 'SDofExp' : std_dev, 
         'MinExp' : min_val,'MaxExp' : max_val, 'RangeOfExp': range_val,
         'IQR':IQR, 'MedianExp':median_val, '25thPerctl':percentile_25, 
         '40thPerctl':percentile_40, '75thPerctl':percentile_75, 'CoeffOfVar': CV,
         'Median_WO_Zeros' : median_wo_zeros
        }
    gene_parameter_df = pd.DataFrame(d)
    
    return gene_parameter_df, gene_exp_dict
        
                
        

In [16]:
gene_parameter_df,gene_exp_dict = gene_level_attributes_function(gtex_shortlisted)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300


In [18]:
print(gene_parameter_df.shape)
gene_parameter_df.head()

(6330, 13)


,Gene,MeanExp,SDofExp,MinExp,MaxExp,RangeOfExp,IQR,MedianExp,25thPerctl,40thPerctl,75thPerctl,CoeffOfVar,Median_WO_Zeros
0,NOC2L,58.911375,27.929395,1.7600,235.400000,233.640000,35.90000,57.36,39.410,50.920,75.31000,0.474092,57.36
1,HES4,26.830241,31.690011,0.0000,567.200000,567.200000,25.54900,18.42,8.451,14.146,34.00000,1.181130,18.43
2,ISG15,59.719838,167.631077,1.4710,7703.000000,7701.529000,40.08000,34.65,17.850,27.566,57.93000,2.806958,34.65
3,ACAP3,71.112203,93.492544,1.4430,1246.000000,1244.557000,51.24000,47.12,28.760,39.150,80.00000,1.314719,47.12
4,PUSL1,16.356359,9.091995,0.5596,85.083933,84.524333,11.30469,14.56,9.747,12.720,21.05169,0.555869,14.56


In [ ]:
gtex_shortlisted_long = gtex_shortlisted.melt(id_vars= 'Gene_name',var_name='SAMPID', value_name='ExpressionVal')
gtex_shortlisted_long['Tissue'] = gtex_shortlisted_long['SAMPID'].map(sampleid_mapping_dict)

def GT(row):
    return row['Gene_name'] +'_'+ row['Tissue']

gtex_shortlisted_long['G_T'] = gtex_shortlisted_long.apply(GT, axis=1)

gtex_shortlisted_long.to_csv("./iPSC/gtex_shortlisted_long_ipsc.csv",index = False)

print(gtex_shortlisted_long.shape)
gtex_shortlisted_long.head()


In [27]:
gene_tissue_expression_dict = {k: list(map(float, v)) for k, v in gtex_shortlisted_long.groupby('G_T')['ExpressionVal']}


KeyboardInterrupt: 

In [ ]:
def counting_ratio(data,thresh,gene):
    total_vals = len(data)
    count = 0
    for i in data:
        try:
            if float(i) >= float(thresh):
                count+=1
        except:
            print(i)
            print(thresh)
            print(gene)
    ratio = count/total_vals
    return ratio

def gene_tissue_level_attributes(gt_exp_dict,g_param_df):
    g_t = []
    gene_list = []
    tissue_list = []
    mean_val = []
    std_dev = []
    min_val = []
    max_val = []
    #range_val = []
    #IQR = []
    median_val = []
    cv = []
    mad = []
    breadth = []
    zscore = []
    count = 0
    for key in gt_exp_dict.keys():
        count+=1
        [gene,tissue] = key.split('_')
        data = gt_exp_dict[key]
        #print(data)
        g_t.append(key)
        gene_list.append(gene)
        tissue_list.append(tissue)
        gt_mean = np.mean(data)
        mean_val.append(gt_mean)
        std_dev.append(np.std(data))
        min_val.append(np.amin(data))
        max_val.append(np.amax(data))
        median_val.append(np.median(data))
        cv.append(np.std(data)/ np.mean(data))
        mad.append(robust.mad(data))
        
        
        #getting data from gene df here
        row = g_param_df[g_param_df['Gene'] == gene]
        gene_mean = row['MeanExp']
        gene_sd = row['SDofExp']
        threshold_breadth = row['Median_WO_Zeros']
        ##done 
        
        breadth.append(counting_ratio(data,threshold_breadth,gene))
        zscore.append(float((gt_mean-gene_mean)/gene_sd))
        if count%10000 == 0:
            print(count)
            
    d = { 'G_T' :g_t,'Gene': gene_list, 'Tissue':tissue_list,
         'MeanExp' : mean_val, 'SDofExp' : std_dev, 'ZScore':zscore,
         'MinExp' : min_val,'MaxExp' : max_val, 'MedianExp':median_val,
         'Breadth': breadth,'CoeffOfVar': cv, 'MedAbsDev':mad
        }
    
    gene_tissue_parameter_df = pd.DataFrame(d)
    
    return gene_tissue_parameter_df
                

In [28]:
len(gene_tissue_expression_dict.keys())

196230

In [ ]:
gene_tissue_parameter_df = gene_tissue_level_attributes(gene_tissue_expression_dict,gene_parameter_df)


In [ ]:
gene_tissue_parameter_df[gene_tissue_parameter_df['Tissue'] == 'iPSC'].head(5)

In [ ]:
len(gene_tissue_parameter_df['Tissue'].drop_duplicates().tolist())

In [ ]:
pickle.dump( gene_tissue_parameter_df, open( "./iPSC/gene_tissue_parameter_df_w_ipsc.p", "wb" ) )

In [29]:
gene_tissue_parameter_df.shape

(196230, 12)

In [30]:
gene_exp_dict_ppi = {}
count = 0
for index,row in gtex_shortlisted.iterrows():
    count+=1
    gene = list(row)[0]
    data = list(row)[1:-1]
    data = np.array(data).astype(np.float)
    gene_exp_dict_ppi[gene] = data
    if count%1000 == 0:
        print(count)

1000
2000
3000
4000
5000
6000


In [31]:
sampleid_mapping_rev_dist = {key:[] for key in set(sampleid_mapping['SMTS'])}
for index,row in sampleid_mapping.iterrows():
    key = row['SMTS']
    val = row ['SAMPID']
    sampleid_mapping_rev_dist[key].append(val)

In [32]:
tf_human  = pd.read_csv('./data/trrust_rawdata.human.tsv',sep='\t',header=None,
                        names = ['TF','Target','ModeofRegulation','PMID'])
tf_human_list = tf_human['TF'].drop_duplicates().tolist()

kinase_list = pd.read_csv('./data/Kincat_Hsap.08.02.csv',index_col=0)
kinase_list = list(kinase_list.index)

In [33]:
gtex_shortlisted = gtex_shortlisted.rename(columns = {'Gene_name':'Gene'})
gtex_genes = gtex_shortlisted['Gene'].tolist()
print(len(gtex_genes))

6330


In [34]:
hippie = pd.read_csv('./data/hippie_current.txt',sep = '\t',header=None)
hippie.columns = ['Gene1', 'Gene1_Entrezid', 'Gene2', 'Gene2_Entrezid' ,'PPI_val', 'Procedure']
hippie = hippie[['Gene1','Gene2','PPI_val']]
# hippie = hippie[hippie['PPI_val']>=0]
hippie = hippie.replace({'_HUMAN':''}, regex=True)
hippie = hippie[(hippie['Gene1'].isin(gtex_genes))&(hippie['Gene2'].isin(gtex_genes))]
print(hippie.shape)
hippie.head(2)

(85298, 3)


,Gene1,Gene2,PPI_val
3,SRGN,CD44,0.63
4,GRB7,ERBB2,0.90


In [35]:
def gene_threshold_dict(gene_list, thresh_percentile):
    gene_threshold_dict = {}
    for gene in gene_list:
        try:
            data = gene_exp_dict_ppi[gene]
            thresh = np.percentile(data,thresh_percentile)
            gene_threshold_dict[gene] = thresh
        except:
            #print(gene + 'gene not found in gtex')
            continue
    return gene_threshold_dict
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def genepair_edge_exp_ratio(g1,g2,exp_df,exp_threshold_dict):
    exp_df = exp_df.set_index('Gene')
    sample_df = exp_df.T
    sample_df_short = sample_df.loc[(sample_df[g1] > exp_threshold_dict[g1]) & (sample_df[g2] > exp_threshold_dict[g2])]
    
    total_vals,x = sample_df.shape
    shortlist_vals,x = sample_df_short.shape
    try:
        ratio = shortlist_vals/total_vals
    except:
        ratio = np.nan
    return ratio

def gene_edge_exp_ratio(exp_df,exp_threshold_dict):
    gene_list = list(exp_df['Gene'])
    print(gene_list[:5])
    exp_df = exp_df.set_index('Gene')
    sample_df = exp_df.T
    gene_edge_exp_ratio_dict = {}
    
    for g in gene_list:
        sample_df_short = sample_df[sample_df[g] > exp_threshold_dict[g]]
        total_vals,x = sample_df.shape
        shortlist_vals,x = sample_df_short.shape
        try:
            ratio = shortlist_vals/total_vals
        except:
            ratio = np.nan
        gene_edge_exp_ratio_dict[g] = ratio
    return gene_edge_exp_ratio_dict

def tissue_specific_exp_data(tissue,samp_gtex_data,sampleid_mapping_rev_dist = sampleid_mapping_rev_dist):
    vals =  sampleid_mapping_rev_dist[tissue]
    gtex_cols = list(samp_gtex_data.columns)
    inters_cols = intersection(vals, gtex_cols)
    inters_cols = ['Gene'] + inters_cols
    return samp_gtex_data[inters_cols]
    
def tissue_network(tissue_name, gene_exp_data=gtex_shortlisted ,ppi_network = hippie,
                   gene_threshold_level = 50, PPI_threshold = 0.01, edge_percentage_threshold = 0.5):
    tissue = tissue_name
    gene_list = list(gene_exp_data['Gene'])
    exp_threshold_dict = gene_threshold_dict(gene_list, gene_threshold_level)
    exp_data_short = tissue_specific_exp_data(tissue_name,gene_exp_data)
    g1_list = []
    g2_list = []
    PPI_val = []
    ratio_val = []
    count = 0
    for index,row in ppi_network.iterrows():
        count+=1
        t1 = row['Gene1']
        t2 = row['Gene2']
        ppi = row['PPI_val']
        gene1 = t1.split('_')[0]
        gene2 = t2.split('_')[0]
        ratio = genepair_edge_exp_ratio(gene1,gene2,exp_data_short,exp_threshold_dict)
        g1_list.append(gene1)
        g2_list.append(gene2)
        PPI_val.append(ppi)
        ratio_val.append(ratio)
        if count%10000 == 0:
            print(count)
            
        tissue_network_df = pd.DataFrame({'Gene1':g1_list,'Gene2':g2_list,
                                     'PPI_val': PPI_val, 'Ratio_val': ratio_val})
    
    gene_edge_exp_ratio_dict = gene_edge_exp_ratio(exp_df = exp_data_short,exp_threshold_dict=exp_threshold_dict)
    tissue_network_df['Gene1_ratio']= tissue_network_df['Gene1'].map(gene_edge_exp_ratio_dict)
    tissue_network_df['Gene2_ratio']= tissue_network_df['Gene2'].map(gene_edge_exp_ratio_dict)
    tissue_network_df['Tissue'] = tissue
    
    tissue_network_df_postthreshold = tissue_network_df[tissue_network_df['Ratio_val'] > edge_percentage_threshold]

    return tissue_network_df_postthreshold

In [36]:
# Clustering is based on this - https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.cluster.clustering.html#networkx.algorithms.cluster.clustering
#
def iftf_func(gene, tf_list = tf_human_list):
    if gene in tf_list:
        return 1
    else:
        return 0
    
def ifkinase_func(gene, kinase_list = kinase_list):
    if gene in kinase_list:
        return 1
    else:
        return 0
    
def tissue_graph_param_function(tissue, tissue_sample_df, ratio_val_threshold = 0.5, tf_list = tf_human_list,kinase_list = kinase_list):
    G = nx.from_pandas_edgelist(tissue_sample_df, 'Gene1', 'Gene2', 'PPI_val')
    clustering_dict = nx.clustering(G)
    sq_clustering_dict = nx.square_clustering(G)
    centrality_dict = nx.betweenness_centrality(G)
    gene_list = list(clustering_dict.keys())
    print(len(gene_list))
    gene = []
    neighbors = []
    is_tf = []
    is_kinase = []
    tf_neighbors = []
    kinase_neighbors = []
    neighbor_list = []
    count = 0
    for g in gene_list:
        count+=1
        gene.append(g)
        neighbors.append(G.degree(g))
        is_tf.append(iftf_func(g))
        is_kinase.append(ifkinase_func(g))
        tf_connections = len(set(tf_list) & set(list(G.neighbors(g))))
        kin_connections = len(set(kinase_list) & set(list(G.neighbors(g))))
        tf_neighbors.append(tf_connections)
        kinase_neighbors.append(kin_connections)
        neighbor_list.append(list(G.neighbors(g)))

        if count%10000 ==0:
            print(count)
            
    tissue_gene_graph_params = pd.DataFrame({'Gene':gene,'NumOfNeighbors':neighbors,
                                            'IsTF':is_tf,'NumOfTFNeighbors':tf_neighbors,
                                             'IsKinase':is_kinase,'NumOfKinaseNbs':kinase_neighbors,
                                            'NeighborList':neighbor_list})
        
    tissue_gene_graph_params['Clustering_coeff']= tissue_gene_graph_params['Gene'].map(clustering_dict)
    tissue_gene_graph_params['Sq_Clustering_coeff']= tissue_gene_graph_params['Gene'].map(sq_clustering_dict)

    tissue_gene_graph_params['Centrality_coeff']= tissue_gene_graph_params['Gene'].map(centrality_dict)
    tissue_gene_graph_params['Tissue'] = tissue
    
    return tissue_gene_graph_params

In [ ]:
tissue_list = ['iPSC']
all_tissue_network_df = pd.DataFrame(columns = ['Gene1', 'Gene2', 'PPI_val', 'Ratio_val', 
                                                'Gene1_ratio', 'Gene2_ratio','Tissue'])
all_tissue_gene_graph_params_df = pd.DataFrame(columns = ['Gene', 'IsTF', 'NumOfNeighbors', 'NumOfTFNeighbors',
       'Clustering_coeff', 'Centrality_coeff','Tissue','IsKinase','NumOfKinaseNbs','NeighborList'])
for tissue in tissue_list:
    print(tissue)
    print(datetime.datetime.now())
    tissue_df = tissue_network(tissue)
    name = tissue + '_network.p'
    print(name)
    pickle.dump( tissue_df, open( './iPSC' + name , "wb" ) )
    tissue_gene_graph_params = tissue_graph_param_function(tissue,tissue_df)    
    all_tissue_network_df = all_tissue_network_df.append(tissue_df,ignore_index=True)
    all_tissue_gene_graph_params_df = all_tissue_gene_graph_params_df.append(tissue_gene_graph_params,ignore_index=True)
    
pickle.dump( all_tissue_network_df, open( './iPSC/all_tissue_network_df_ipsc.p' , "wb" ) )
pickle.dump( all_tissue_gene_graph_params_df, open( './iPSC/all_tissue_gene_graph_params_df_ipsc.p' , "wb" ) )
    

iPSC
2020-07-28 22:38:14.145857
10000
20000
30000
40000
50000
60000
70000
80000
['NOC2L', 'HES4', 'ISG15', 'ACAP3', 'PUSL1']
iPSC_network.p


In [39]:
print(all_tissue_gene_graph_params_df.shape)
all_tissue_gene_graph_params_df.head(4)


(3834, 11)


,Centrality_coeff,Clustering_coeff,Gene,IsKinase,IsTF,NeighborList,NumOfKinaseNbs,NumOfNeighbors,NumOfTFNeighbors,Sq_Clustering_coeff,Tissue
0,0.000147,0.125000,GRB7,0,0,"[ERBB2, KIT, ERBB3, GRB2, RND1, SHC1, RET, EPH...",3,19,1,0.002603,iPSC
1,0.005954,0.077434,ERBB2,0,0,"[GRB7, PAK1, DLG4, MUC4, PICK1, GRB2, TGFA, ER...",16,115,7,0.001435,iPSC
2,0.001194,0.071373,PAK1,1,0,"[ERBB2, PLCG1, LIMK1, RAC1, PAK1, HGS, NCK2, F...",9,53,8,0.001082,iPSC
3,0.002871,0.076655,DLG4,0,0,"[ERBB2, LYN, PTPRG, DLG3, NLGN3, LRP2, GRIK5, ...",3,44,2,0.001025,iPSC


In [40]:
all_tissue_gene_graph_params_df.shape

(3834, 11)